In [1]:
from human_eval.execution import check_correctness
from human_eval.data import HUMAN_EVAL, stream_jsonl
import numpy as np

In [6]:
problems = list(stream_jsonl(HUMAN_EVAL))
tid_map = {problem["task_id"]: problem for problem in problems}

In [72]:
idx = 142
problem = problems[idx]
print(problem['prompt'])





def sum_squares(lst):
    """"
    This function will take a list of integers. For all entries in the list, the function shall square the integer entry if its index is a 
    multiple of 3 and will cube the integer entry if its index is a multiple of 4 and not a multiple of 3. The function will not 
    change the entries in the list whose indexes are not a multiple of 3 or 4. The function shall then return the sum of all entries. 
    
    Examples:
    For lst = [1,2,3] the output should be 6
    For lst = []  the output should be 0
    For lst = [-1,-5,2,-1,-5]  the output should be -126
    """



In [63]:
print(problem['canonical_solution'])
print(problem["test"])

    result =[]
    for i in range(len(lst)):
        if i %3 == 0:
            result.append(lst[i]**2)
        elif i % 4 == 0 and i%3 != 0:
            result.append(lst[i]**3)
        else:
            result.append(lst[i])
    return sum(result)

def check(candidate):

    # Check some simple cases
    
    assert candidate([1,2,3]) == 6
    assert candidate([1,4,9]) == 14
    assert candidate([]) == 0
    assert candidate([1,1,1,1,1,1,1,1,1]) == 9
    assert candidate([-1,-1,-1,-1,-1,-1,-1,-1,-1]) == -3
    assert candidate([0]) == 0
    assert candidate([-1,-5,2,-1,-5]) == -126
    assert candidate([-56,-99,1,0,-2]) == 3030
    assert candidate([-1,0,0,0,0,0,0,0,-1]) == 0
    assert candidate([-16, -9, -2, 36, 36, 26, -20, 25, -40, 20, -4, 12, -26, 35, 37]) == -14196
    assert candidate([-1, -3, 17, -1, -15, 13, -1, 14, -14, -12, -5, 14, -14, 6, 13, 11, 16, 16, 4, 10]) == -1448
    
    
    # Don't remove this line:



In [8]:
## TODO: should be returning fraction of test cases passed, instead of 0/1
def execute(problem, completion, timeout=10):
    result = check_correctness(problem, completion, timeout)
    return result


In [67]:
problem = problems[163]
completion = "\n    return [i for i in range(min(a, b), max(a, b)+1) if i % 2 == 0]\n\n"
print(completion)


    return [i for i in range(min(a, b), max(a, b)+1) if i % 2 == 0]




In [74]:
execute(problem, completion)

{'task_id': 'HumanEval/142',
 'passed': False,
 'result': "failed: name 'a' is not defined",
 'completion_id': None}

In [75]:
incorrect_completion = "    return False"
execute(problem, incorrect_completion)

{'task_id': 'HumanEval/142',
 'passed': False,
 'result': 'failed: ',
 'completion_id': None}

In [84]:
mcts_solutions = list(stream_jsonl("../few_shot_mcts.jsonl"))
mcts_results = []
for solution in mcts_solutions:
    problem = tid_map[solution["task_id"]]
    completion = solution["completion"]
    result = execute(problem, completion)
    mcts_results.append(result["passed"])

Total pass rate: 92.59%


In [92]:
mcts_pass_rate = 100 * sum(mcts_results) / len(mcts_results)
mcts_total_gens = sum([solution["stats"]["num_generations"] for solution in mcts_solutions])
mcts_mean_gens = mcts_total_gens / len(mcts_solutions)
mcts_mean_rollouts = sum([solution["stats"]["num_rollouts"] for solution in mcts_solutions]) / len(mcts_solutions)
print(f"MCTS pass rate: {mcts_pass_rate:.2f}%")
print(f"Avg num generations: {mcts_mean_gens:.2f}")
print(f"Avg num rollouts: {mcts_mean_rollouts:.2f}")

MCTS pass rate: 92.59%
Avg num generations: 3.62
Avg num rollouts: 15.47


In [4]:
baseline_solutions = list(stream_jsonl("../few_shot_baselines_256_top_3.jsonl"))
tid_completions_map = {}
for solution in baseline_solutions:
    tid_completions_map.setdefault(solution["task_id"], []).append(solution["completion"])

In [29]:
baseline_results = []
for task_id, completions in tid_completions_map.items():
    problem = tid_map[task_id]
    result = False
    for completion in completions:
        r = execute(problem, completion)
        if r["passed"]:
            result = True
            break
    baseline_results.append(result)

In [41]:
baseline_pass_rate = 100 * sum(baseline_results) / len(baseline_results)
baseline_k = len(baseline_solutions) / len(baseline_results)
print(f"Baseline pass@{int(baseline_k)}: {baseline_pass_rate:.2f}%")

Baseline pass@20: 90.74%


In [42]:
baseline_results = []
for task_id, completions in tid_completions_map.items():
    problem = tid_map[task_id]
    result = False
    for completion in completions[:5]:
        r = execute(problem, completion)
        if r["passed"]:
            result = True
            break
    baseline_results.append(result)

In [43]:
baseline_pass_rate = 100 * sum(baseline_results) / len(baseline_results)
print(f"Baseline pass@5: {baseline_pass_rate:.2f}%")

Baseline pass@5: 85.80%
